In [1]:
# https://towardsdatascience.com/adversarial-machine-learning-mitigation-adversarial-learning-9ae04133c137
import cleverhans
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from keras.utils import np_utils
from cleverhans.future.tf2.attacks import fast_gradient_method

In [2]:
# new Data preparation

# Training Dataset
data = pd.read_csv("./dataset/UNSW_2018_IoT_Botnet_Final_10_best_Training.csv")

x_features = data[['seq','stddev','N_IN_Conn_P_SrcIP', 'min', 'state_number', 'mean', 'N_IN_Conn_P_DstIP',
       'drate', 'srate', 'max', 'category', 'subcategory']]

# Label Encoding the target columns
le = LabelEncoder()
x_features['category'] = le.fit_transform(x_features['category'])
x_features['subcategory'] = le.fit_transform(x_features['subcategory'])

x = x_features.values
x = preprocessing.scale(x)
y = data['attack'].values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

y_train_cat = np_utils.to_categorical(y_train)
y_test_cat = np_utils.to_categorical(y_test)

C:\Users\sumsu\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\sumsu\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [3]:
model = tf.keras.Sequential([
        tf.keras.layers.Dense(200, input_dim=x.shape[1], activation=tf.nn.relu),
        tf.keras.layers.Dense(500, activation=tf.nn.relu),
        tf.keras.layers.Dense(200, activation=tf.nn.relu),
        tf.keras.layers.Dense(len(np.unique(y))),
        # We seperate the activation layer to be able to access
        # the logits of the previous layer later
        tf.keras.layers.Activation(tf.nn.softmax)
        ])

model.compile(optimizer='adam',
              loss= 'categorical_crossentropy',
              metrics=['accuracy'])

In [4]:
model.fit(X_train, y_train_cat, epochs=10, validation_split=0.2)
test_loss, test_acc = model.evaluate(X_test, y_test_cat)
print('Test accuracy:', test_acc)

Epoch 1/10
58697/58697 [==============================] - 492s 8ms/step - loss: 0.0039 - accuracy: 0.9999 - val_loss: 2.6019e-04 - val_accuracy: 1.0000
Epoch 2/10
58697/58697 [==============================] - 413s 7ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 3.3653e-05 - val_accuracy: 1.0000
Epoch 3/10
58697/58697 [==============================] - 386s 7ms/step - loss: 4.8151e-04 - accuracy: 1.0000 - val_loss: 7.6270e-05 - val_accuracy: 1.0000
Epoch 4/10
58697/58697 [==============================] - 389s 7ms/step - loss: 4.8743e-04 - accuracy: 1.0000 - val_loss: 7.4880e-07 - val_accuracy: 1.0000
Epoch 5/10
58697/58697 [==============================] - 412s 7ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 3.0771e-07 - val_accuracy: 1.0000
Epoch 6/10
58697/58697 [==============================] - 320s 5ms/step - loss: 3.6863e-04 - accuracy: 1.0000 - val_loss: 2.7552e-06 - val_accuracy: 1.0000
Epoch 7/10
58697/58697 [==============================] - 188s 3ms/step - lo

In [5]:
y_pred = model.predict(X_test)
y_pred=np.argmax(y_pred, axis=1)
cm_org = confusion_matrix(y_test, y_pred)
print("*"*50)
print("Original confusion matrix")
print(cm_org)

18343/18343 [==============================] - 21s 1ms/step
**************************************************
Original confusion matrix
[[    66      0]
 [     2 586896]]


In [6]:
# Import the attack
#The attack requires the model to ouput the logits
logits_model = tf.keras.Model(model.input, model.layers[-1].output)

epsilon = 0.1
adv_fgsm_x = fast_gradient_method(logits_model,
                                  X_test,
                                  epsilon,
                                  np.inf,
                                  targeted=False)

In [7]:
y_pred = model.predict(adv_fgsm_x)
y_pred=np.argmax(y_pred, axis=1)
cm_adv = confusion_matrix(y_test, y_pred)
print("*"*20)
print("Attacked confusion matrix")
print(cm_adv)

18343/18343 [==============================] - 25s 1ms/step
********************
Attacked confusion matrix
[[    66      0]
 [     2 586896]]


In [8]:
print(adv_fgsm_x)

tf.Tensor(
[[ 0.23291182  1.25152354  0.71545303 ...  0.74496694 -0.83068184
  -0.86685956]
 [-0.88787428 -0.70547208  0.71545303 ...  0.50834781  0.75246724
   0.84493152]
 [-0.35340257  0.60108472 -2.15455363 ...  1.00122964 -0.83068184
   0.84493152]
 ...
 [ 1.02034253  0.86647627 -1.00655096 ...  0.40499218  0.75246724
   0.84493152]
 [-1.41646107  0.44483421  0.71545303 ... -0.15993703  0.75246724
   0.84493152]
 [-0.27933972  1.22739828  0.71545303 ...  1.03711469  0.75246724
   0.84493152]], shape=(586964, 12), dtype=float64)
